<a href="https://colab.research.google.com/github/Buuuuli/AIPI540_recommendation/blob/main/DL_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT, force_remount=False)

Mounted at /content/drive


In [2]:
%cd /content/drive/"My Drive"/AIPI540_recom/data

/content/drive/My Drive/AIPI540_recom/data


In [6]:
import sys
import os
import torch
import cornac
import papermill as pm
import scrapbook as sb
import pandas as pd
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_random_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.timer import Timer
from recommenders.utils.constants import SEED

print("System version: {}".format(sys.version))
print("PyTorch version: {}".format(torch.__version__))
print("Cornac version: {}".format(cornac.__version__))

System version: 3.7.13 (default, Mar 16 2022, 17:37:17) 
[GCC 7.5.0]
PyTorch version: 1.10.0+cu111
Cornac version: 1.14.2


In [7]:
df = pd.read_pickle('final_dataframe.pkl')

In [8]:
df_new = df.sample(20000)

In [9]:
data_encoded = df_new.copy()
for col in ['business_id','user_id']:
    data_encoded[col] = data_encoded[col].astype('category') # Convert to category type
    data_encoded[col] = data_encoded[col].cat.codes # Convert to numerical code
data_encoded.head()

,business_id,name,address,city,state,postal_code,stars,review_count,categories,user_id,review_stars
123732,4398,Bar Louie - Tampa,2223 N Westshore Blvd,Tampa,FL,33607,4.0,451,"Cocktail Bars, American (New), Restaurants, Bu...",15388,4
562737,2837,Park Square Social,"16132 Churchview Dr, Ste 101",Lithia,FL,33547,3.5,42,"Restaurants, Beer, Wine & Spirits, Waffles, Ni...",2676,5
441181,1901,Clear Sky Cafe,"490 Mandalay Ave, Ste 1",Clearwater,FL,33767,4.5,1824,"Restaurants, American (Traditional), Tapas/Sma...",11790,4
564310,3017,Splitsville Southern + Social,"615 Channelside Dr, Ste 120",Tampa,FL,33602,3.5,43,"Restaurants, Southern, Wine Bars, Bars, Event ...",11442,4
452754,855,Zaytoon Grill,1618 Central Ave,Saint Petersburg,FL,33710,4.5,240,"Mediterranean, Restaurants",8567,5


In [10]:
data_encoded_only = data_encoded[['business_id','user_id','review_stars']]

In [11]:
data_encoded_only = data_encoded_only.reset_index(drop=True)

In [12]:
data_encoded_only

,business_id,user_id,review_stars
0,4398,15388,4
1,2837,2676,5
2,1901,11790,4
3,3017,11442,4
4,855,8567,5
...,...,...,...
19995,2228,13275,5
19996,236,8669,1
19997,3258,1902,5
19998,2329,14793,5


In [13]:
data_encoded_only.rename(columns = {'business_id':'itemID', 'user_id':'userID','review_stars':'rating'}, inplace = True)

In [14]:
train, test = python_random_split(data_encoded_only, 0.75)

In [15]:
# top k items to recommend
TOP_K = 10

# Model parameters
LATENT_DIM = 5
ENCODER_DIMS = [300]
ACT_FUNC = "tanh"
LIKELIHOOD = "gaus"
NUM_EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [16]:
train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False), seed=SEED)

print('Number of users: {}'.format(train_set.num_users))
print('Number of items: {}'.format(train_set.num_items))

Number of users: 4146
Number of items: 12586


/usr/local/lib/python3.7/dist-packages/cornac/data/dataset.py:361: UserWarning: 18 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [17]:
bivae = cornac.models.BiVAECF(
    k=LATENT_DIM,
    encoder_structure=ENCODER_DIMS,
    act_fn=ACT_FUNC,
    likelihood=LIKELIHOOD,
    n_epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    seed=SEED,
    use_gpu=torch.cuda.is_available(),
    verbose=True
)

with Timer() as t:
    bivae.fit(train_set)
print("Took {} seconds for training.".format(t))

  0%|          | 0/500 [00:00<?, ?it/s]

Took 1142.5991 seconds for training.


In [18]:
with Timer() as t:
    all_predictions = predict_ranking(bivae, train, usercol='userID', itemcol='itemID', remove_seen=True)
print("Took {} seconds for prediction.".format(t))

Took 71.1042 seconds for prediction.


In [19]:
all_predictions

,userID,itemID,prediction
15000,4198,956,0.001825
15001,4198,3628,0.001383
15002,4198,7931,0.001558
15003,4198,13224,0.002100
15004,4198,5412,0.001236
...,...,...,...
52193726,64,1881,0.001080
52193727,64,1685,0.001579
52193728,64,7844,0.001465
52193729,64,14235,0.001447


In [20]:
eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.000083
NDCG:	0.000239
Precision@K:	0.000083
Recall@K:	0.000827


In [21]:
%cd /content/drive/"My Drive"/AIPI540_recom/data

/content/drive/My Drive/AIPI540_recom/data


In [22]:

filename = 'bivae_model.pt'

# Save the entire model
torch.save(bivae, filename)

In [23]:
model = torch.load(filename)

c = predict_ranking(model, train, usercol='userID', itemcol='itemID', remove_seen=True)
